# Experiment Notebook: {Descriptive Experiment Keywords}

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to...

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [53]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

time: 53.5 ms (started: 2023-02-07 18:28:26 +08:00)


In [54]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

time: 14.9 ms (started: 2023-02-07 18:28:26 +08:00)


In [55]:
# Import experiment templates
import experiments.templates.polygon.staked as phase_analysis_over_stake
import experiments.templates.polygon.price as phase_analysis_over_price
import experiments.templates.polygon.issuance as phase_analysis_over_issuance_method

time: 14.9 ms (started: 2023-02-07 18:28:26 +08:00)


In [56]:
# Inspect experiment template
display_code(phase_analysis_over_price)

"""
# ETH Price Sweep Analysis

Creates a parameter sweep of the ETH price process,
with a static value for ETH staked set to the current ETH staked value from Beaconcha.in.
"""

import numpy as np
import copy

from model.state_variables import polygn_staked
from model.types import Stage
from experiments.default_experiment import experiment, TIMESTEPS, DELTA_TIME
import model.constants as constants


DELTA_TIME = 7*constants.epochs_per_day  # epochs per timestep
SIMULATION_TIME_MONTHS = 12 * 10  # number of months
TIMESTEPS = constants.epochs_per_month * SIMULATION_TIME_MONTHS // DELTA_TIME

# Make a copy of the default experiment to avoid mutation
experiment = copy.deepcopy(experiment)

# ETH price range from 100 USD/ETH to the maximum over the last 12 months
polygn_price_samples = np.linspace(start=0.5, stop=10, num=50)

PoS_checkpoint_gas_cost = constants.PoS_checkpoint_gas_cost
parameter_overrides = {
    "polygn_price_process": [
        lambda run, _timestep: polygn_price_samples[run - 1]
    ],
    "polygn_staked_process": [
        lambda _run, _timestep: polygn_staked,
    ],
    "Adoption_speed": [0],
    "checkpoint_gas_cost": [0.1 * PoS_checkpoint_gas_cost]
}

# Override default experiment parameters
experiment.simulations[0].model.params.update(parameter_overrides)
# Set runs to number of combinations in sweep
experiment.simulations[0].runs = len(polygn_price_samples)
# Run single timestep, set unit of time to multiple epochs
experiment.simulations[0].timesteps = 1
experiment.simulations[0].model.params.update({"dt": [ TIMESTEPS * DELTA_TIME]})

time: 27.9 ms (started: 2023-02-07 18:28:26 +08:00)


In [57]:
# gas cost comsumption and validator yield analysis
simulation_1 = copy.deepcopy(phase_analysis_over_stake.experiment.simulations[0])

time: 17.3 ms (started: 2023-02-07 18:28:26 +08:00)


In [58]:
simulation_1.model.initial_state.update({
    "PUBLIC_CHAINS_CNT": 5,
    "PRIVATE_CHAINS_CNT": 1000,
    "stake_data_onchain": False,
})

time: 15.1 ms (started: 2023-02-07 18:28:26 +08:00)


In [59]:
df_1, _exceptions = run(simulation_1)

2023-02-07 18:28:26,412 - root - INFO - Running experiment
2023-02-07 18:28:26,416 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-02-07 18:28:26,429 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-02-07 18:28:26,435 - root - INFO - Starting simulation 0 / run 0 / subset 2
2023-02-07 18:28:26,440 - root - INFO - Starting simulation 0 / run 0 / subset 3
2023-02-07 18:28:26,446 - root - INFO - Starting simulation 0 / run 1 / subset 0
2023-02-07 18:28:26,452 - root - INFO - Starting simulation 0 / run 1 / subset 1
2023-02-07 18:28:26,457 - root - INFO - Starting simulation 0 / run 1 / subset 2
2023-02-07 18:28:26,463 - root - INFO - Starting simulation 0 / run 1 / subset 3
2023-02-07 18:28:26,468 - root - INFO - Starting simulation 0 / run 2 / subset 0
2023-02-07 18:28:26,475 - root - INFO - Starting simulation 0 / run 2 / subset 1
2023-02-07 18:28:26,481 - root - INFO - Starting simulation 0 / run 2 / subset 2
2023-02-07 18:28:26,486 - root - INFO - Starting s

In [60]:
visualizations.plot_revenue_profit_yields_over_polygn_staked(df_1)

time: 29.8 ms (started: 2023-02-07 18:28:27 +08:00)


In [61]:
#df_1[["total_profit","validator_costs","validator_checkpoint_costs","chain_specific_checkpoint_submission_cadence"]]

time: 14.8 ms (started: 2023-02-07 18:28:27 +08:00)


In [62]:
## price sweep analysis
simulation_2 = copy.deepcopy(phase_analysis_over_price.experiment.simulations[0])

simulation_2.model.initial_state.update({
    "PUBLIC_CHAINS_CNT": 5,
    "PRIVATE_CHAINS_CNT": 1000,
    "stake_data_onchain": True,
})

df_2, _exceptions = run(simulation_2)

2023-02-07 18:28:27,674 - root - INFO - Running experiment
2023-02-07 18:28:27,676 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-02-07 18:28:27,681 - root - INFO - Starting simulation 0 / run 1 / subset 0
2023-02-07 18:28:27,686 - root - INFO - Starting simulation 0 / run 2 / subset 0
2023-02-07 18:28:27,692 - root - INFO - Starting simulation 0 / run 3 / subset 0
2023-02-07 18:28:27,697 - root - INFO - Starting simulation 0 / run 4 / subset 0
2023-02-07 18:28:27,702 - root - INFO - Starting simulation 0 / run 5 / subset 0
2023-02-07 18:28:27,707 - root - INFO - Starting simulation 0 / run 6 / subset 0
2023-02-07 18:28:27,712 - root - INFO - Starting simulation 0 / run 7 / subset 0
2023-02-07 18:28:27,717 - root - INFO - Starting simulation 0 / run 8 / subset 0
2023-02-07 18:28:27,722 - root - INFO - Starting simulation 0 / run 9 / subset 0
2023-02-07 18:28:27,727 - root - INFO - Starting simulation 0 / run 10 / subset 0
2023-02-07 18:28:27,732 - root - INFO - Starting 

In [63]:
visualizations.plot_revenue_profit_yields_over_polygn_price(df_2)

time: 21.4 ms (started: 2023-02-07 18:28:27 +08:00)


In [64]:
## price sweep analysis
simulation_3 = copy.deepcopy(phase_analysis_over_issuance_method.experiment.simulations[0])

simulation_3.model.initial_state.update({
    "PUBLIC_CHAINS_CNT": 5,
    "PRIVATE_CHAINS_CNT": 1000,
    "stake_data_onchain": False,
})

df_3, _exceptions = run(simulation_3)

2023-02-07 18:28:28,055 - root - INFO - Running experiment
2023-02-07 18:28:28,057 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-02-07 18:28:28,062 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-02-07 18:28:28,067 - root - INFO - Starting simulation 0 / run 1 / subset 0
2023-02-07 18:28:28,072 - root - INFO - Starting simulation 0 / run 1 / subset 1
2023-02-07 18:28:28,077 - root - INFO - Starting simulation 0 / run 2 / subset 0
2023-02-07 18:28:28,083 - root - INFO - Starting simulation 0 / run 2 / subset 1
2023-02-07 18:28:28,087 - root - INFO - Starting simulation 0 / run 3 / subset 0
2023-02-07 18:28:28,092 - root - INFO - Starting simulation 0 / run 3 / subset 1
2023-02-07 18:28:28,097 - root - INFO - Starting simulation 0 / run 4 / subset 0
2023-02-07 18:28:28,103 - root - INFO - Starting simulation 0 / run 4 / subset 1
2023-02-07 18:28:28,108 - root - INFO - Starting simulation 0 / run 5 / subset 0
2023-02-07 18:28:28,113 - root - INFO - Starting s

In [65]:
visualizations.plot_revenue_profit_yields_over_issuance_curves(df_3)

time: 27 ms (started: 2023-02-07 18:28:28 +08:00)
